In [14]:
import pandas as pd
import numpy as np
import altair as alt
from ipywidgets import interact
from os.path import join

from constants import COLUMNS
from utils import read_latest_demographics_df

In [15]:
df = read_latest_demographics_df()
SITE_IDS = df[COLUMNS.SITE_ID].unique().tolist()

# Columns
siteid = COLUMNS.SITE_ID
sex = COLUMNS.SEX
total_patients = COLUMNS.TOTAL_PATIENTS

# remove aggregate rows and columns
not_all_filter = df['sex']!="All"
df = df[not_all_filter]
df = df.drop(columns=['total_patients'])

# wide to long
df = pd.melt(df, id_vars=['siteid','sex'])
df = df.rename(columns={"variable": "age_group", "value": "num_patients"})

df.head()

,siteid,sex,age_group,num_patients
0,FZU,Male,age_0to2,1
1,FZU,Female,age_0to2,4
2,FZU,Other,age_0to2,0
3,FZT,Male,age_0to2,21
4,FZT,Female,age_0to2,7


## Summarization Chart
##### Select Site ID to see data by a specific site ID

In [16]:
sex_color_scale = alt.Scale(domain=["Male", "Female", "Other"], range=["#0072B2", "#D55E00", "gray"])

def apply_theme(base):
    return base.configure_axis(
        labelFontSize=14,
        labelFontWeight=300,
        titleFontSize=18,
        titleFontWeight=300
    ).configure_title(fontSize=18, fontWeight=400, anchor="middle")
    
def demographics_chart(SiteID, Normalize): 
    
    # Base bar chart
    base = alt.Chart(df).mark_bar().encode(
        x=alt.X('age_group:N', title="Age Group",sort="x"),
        y=alt.Y(f"sum(num_patients):Q", title="Number of Patients", axis=alt.Axis(tickCount=5)),
        
        color=alt.Color("sex:N", title="Sex", scale=sex_color_scale),
        tooltip=[siteid, sex, f"sum(num_patients)"]
    ).properties(
        title="COVID-19 Patients (" + SiteID + ")",
        width=500,
        height=300
    )
    
    if SiteID != "All Sites":
        base = base.transform_filter(
            alt.FieldEqualPredicate(field=siteid, equal=SiteID)
        )

    if Normalize != "no":
        base = base.encode(
            y=alt.Y(f"sum(num_patients):Q", title="Fraction of Patients", stack="normalize"),
        )    
    
    chart = apply_theme(base)
        
    return chart.interactive()

interact(demographics_chart, SiteID=["All Sites"] + SITE_IDS, Normalize=["yes", "no"] )

interactive(children=(Dropdown(description='SiteID', options=('All Sites', 'FZU', 'FZT', 'FZM', 'FXL', 'FWN', …

<function __main__.demographics_chart(SiteID, Normalize)>

In [17]:
click = alt.selection_multi(encodings=['color'])

def apply_theme(base):
    return base.configure_axis(
        labelFontSize=14,
        labelFontWeight=300,
        titleFontSize=18,
        titleFontWeight=300
    ).configure_title(fontSize=18, fontWeight=400, anchor="start")
    
def demographics_chart():     
    base = alt.Chart(df).mark_bar().encode(
        x=alt.X('siteid:N', title="Site",sort="x", axis=None),
        y=alt.Y(f"sum(num_patients):Q", title="Number of Patients", axis=alt.Axis(tickCount=5)),
        column=alt.Column('age_group:O', title="", sort=['age_0to2', 
                                                         'age_3to5',
                                                         'age_6to11',
                                                         'age_12to17', 
                                                         'age_18to25',
                                                         'age_26to49',
                                                         'age_50to69',
                                                         'age_70to79',
                                                         'age_80plus']),
        
        color=alt.Color("siteid:N", title="Site", scale=alt.Scale(scheme="category20"), legend=None),
        tooltip=[siteid, f"sum(num_patients)"]
    ).properties(
        width=70,
        height=300
    ).transform_filter(
        click
    )
        
    return base.interactive()

def site_chart():     
    base = alt.Chart(df).mark_circle(size=100).encode(
        y=alt.Y('siteid:N', title="Site",sort="y"),
        color=alt.condition( click, alt.Color("siteid:N", title="Site", scale=alt.Scale(scheme="category20"), legend=None), alt.value('gray') ),
    ).properties(
        selection=click        
    ).properties(
        title="",
        height=300
    )
                
    return base.interactive()



apply_theme( site_chart() | demographics_chart()
    ).properties(
        title="COVID-19 Patients by Age Group and Site"
    )

alt.HConcatChart(...)

Removing narrow white strokes arround bars by not separating bars by `siteid` when "All Sites" is selected.

In [49]:
sex_color_scale = alt.Scale(domain=["Male", "Female", "Other"], range=["#0072B2", "#D55E00", "gray"])

def apply_theme(base):
    return base.configure_axis(
        labelFontSize=14,
        labelFontWeight=300,
        titleFontSize=18,
        titleFontWeight=300
    ).configure_title(fontSize=18, fontWeight=400, anchor="middle")
    
def demographics_chart(SiteID, Normalize): 
    
    # Base bar chart
    base = alt.Chart(df).mark_bar().encode(
        x=alt.X('age_group:N', title="Age Group",sort="x"),
        y=alt.Y(f"sum(num_patients):Q", title="Number of Patients", axis=alt.Axis(tickCount=5)),
        
        color=alt.Color("sex:N", title="Sex", scale=sex_color_scale),
    ).properties(
        title="COVID-19 Patients (" + SiteID + ")",
        width=500,
        height=300
    )
    
    if SiteID != "All Sites":
        base = base.transform_filter(
            alt.FieldEqualPredicate(field=siteid, equal=SiteID)
        ).encode(tooltip=[siteid, sex, f"sum(num_patients)"])
    else:
        base = base.encode(tooltip=[sex, f"sum(num_patients)"])

    if Normalize != "no":
        base = base.encode(
            y=alt.Y(f"sum(num_patients):Q", title="Fraction of Patients", stack="normalize"),
        )
    
    chart = apply_theme(base)
        
    return (chart).interactive()

interact(demographics_chart, SiteID=["All Sites"] + SITE_IDS, Normalize=["yes", "no"] )

interactive(children=(Dropdown(description='SiteID', options=('All Sites', 'FZU', 'FZT', 'FZM', 'FXL', 'FWN', …

<function __main__.demographics_chart(SiteID, Normalize)>